In [2]:
import re
import os

from jax_fem.generate_mesh import get_meshio_cell_type,box_mesh,Mesh

from scipy.spatial.transform import Rotation

from Library.Plasticity import Plasticity
from Library.create_press import create_flat_rect_press

import time

from vtk_plotting import create_gif

In [3]:
import jax.numpy as np
from jax_fem.solver import solver
from jax_fem.utils import save_sol

class PlasticitySim:
    def __init__(self,problem,presses,data_dir):
        self.problem = problem
        self.presses = presses
        self.data_dir = data_dir
        self.sol_list = [np.zeros_like(problem.mesh[0].points)]
        self.prev_sol_list = [np.copy(self.sol_list[0])]
        self.res_vals = []
        self.avg_stresses = []
        self.step = 0

    def get_dirichlet_bc_info_and_predicted_displacements(self):
        mesh_points = self.problem.mesh[0].points
        predicted_displacements = np.zeros_like(mesh_points)
        location_functions=[];vecs=[];value_functions=[]
        for press in self.presses:
            checks,displacements = press.get_checks_and_displacements(self.problem,self.sol_list,self.prev_sol_list)

            press_dirichlet_bc_info = press.get_dirichlet_bc_info(mesh_points,checks,self.sol_list[0]+displacements)
            location_functions += press_dirichlet_bc_info[0]
            vecs += press_dirichlet_bc_info[1]
            value_functions += press_dirichlet_bc_info[2]

            predicted_displacements += displacements

        return [location_functions,vecs,value_functions],predicted_displacements

    def run_simulation_step(self,problem,dirichlet_bc_info,initial_guess,save_path=False):
        problem.fe.update_Dirichlet_boundary_conditions(dirichlet_bc_info)
        sol_list,res_val = solver(problem,initial_guess=initial_guess)
        #problem.update_stress_strain(sol_list[0])
        avg_stress = problem.compute_avg_stress()
        if save_path: save_sol(problem.fe,sol_list[0],save_path)
        return problem,sol_list,res_val,avg_stress
    
    def update(self,dirichlet_bc_info=False,predicted_displacements=False):
        save_path = os.path.join(self.data_dir,f'u_{self.step+1:03d}.vtu')

        start1 = time.time()
        if not dirichlet_bc_info:
            dirichlet_bc_info,predicted_displacements = self.get_dirichlet_bc_info_and_predicted_displacements()
        print(f"DirichletBCInfo: {time.time()-start1}")

        initial_guess = [self.sol_list[0]+predicted_displacements]

        start2 = time.time()
        problem,sol_list,res_val,avg_stress = self.run_simulation_step(self.problem,dirichlet_bc_info,initial_guess,save_path)
        print(f"Simulation: {time.time()-start2}")

        if res_val <= 1:
            self.problem = problem
            self.prev_sol_list = self.sol_list
            self.sol_list = sol_list
            for press in self.presses:
                press.update()
            self.step += 1

            #print("SOL_LIST")
            #print(np.sum(abs(sol_list[0])))

        self.res_vals.append(res_val)
        self.avg_stresses.append(avg_stress)


In [4]:
# SETUP/INITIALIZATION FUNCTIONS
def create_next_run_folder(directory):
    pattern = re.compile(r"^run_(\d+)$")
    items = os.listdir(directory)
    run_folders = [item for item in items if os.path.isdir(os.path.join(directory, item)) and pattern.match(item)]
    run_folders.sort(key=lambda x: int(pattern.match(x).group(1)))
    
    if run_folders:
        last_run_number = int(pattern.match(run_folders[-1]).group(1))
        next_run_number = last_run_number + 1
    else:
        next_run_number = 1
    
    next_run_folder = f"run_{next_run_number:03d}"
    next_run_folder_path = os.path.join(directory, next_run_folder)
    os.makedirs(next_run_folder_path)
    run_folders.append(next_run_folder)

    return run_folders, next_run_folder_path

def setup_data_dir(outputFolder='data/runs'):
    if not os.path.exists(outputFolder):
        os.makedirs(outputFolder)
    _,run_path = create_next_run_folder(outputFolder)
    return run_path

def setup_starting_mesh(data_dir):
    # Specify mesh-related information(first-order hexahedron element).
    ele_type = 'HEX8'
    cell_type = get_meshio_cell_type(ele_type)
    Lx, Ly, Lz = 2., 2., 2.
    meshio_mesh = box_mesh(Nx=9, Ny=9, Nz=9, Lx=Lx, Ly=Ly, Lz=Lz, data_dir=data_dir, ele_type=ele_type)
    mesh = Mesh(meshio_mesh.points, meshio_mesh.cells_dict[cell_type])
    # ADD NOISE
    #noiseArr = onp.random.normal(0,0.01,mesh.points.shape)
    #mesh.points = mesh.points + noiseArr
    # ROTATE MESH
    rot = Rotation.from_euler('x',50,degrees=True)
    mesh.points = rot.apply(mesh.points)
    rot = Rotation.from_euler('y',10,degrees=True)
    mesh.points = rot.apply(mesh.points)
    #mesh.points = rotate_points(mesh.points, [0,1,0], onp.degrees(onp.arctan(onp.sqrt(2)/2)), [Lx/2,Ly/2,Lz/2])
    return mesh


In [5]:
def get_rectangular_hit_sim(size,offset,hit_depth,mesh,data_dir,step_size=0.03):
    problem = Plasticity(mesh,vec=3,dim=3)
    press = create_flat_rect_press(size,offset,[0.,0.,-step_size],mesh.points,vecs=[0,1,2])
    table = create_flat_rect_press([10.,10.],[0.,0.],[0.,0.,1e-6],mesh.points,vecs=[0,1,2],isTable=True)
    table.update_displacement([0.,0.,0.])
    num_steps = int(min(abs(hit_depth/step_size),100))

    return PlasticitySim(problem,[press,table],data_dir),num_steps


In [6]:
data_dir = setup_data_dir()
mesh = setup_starting_mesh(data_dir)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Extruded)
Info    : [ 10%] Meshing curve 2 (Extruded)
Info    : [ 20%] Meshing curve 3 (Extruded)
Info    : [ 30%] Meshing curve 4 (Extruded)
Info    : [ 40%] Meshing curve 7 (Extruded)
Info    : [ 50%] Meshing curve 8 (Extruded)
Info    : [ 50%] Meshing curve 9 (Extruded)
Info    : [ 60%] Meshing curve 10 (Extruded)
Info    : [ 70%] Meshing curve 12 (Extruded)
Info    : [ 80%] Meshing curve 13 (Extruded)
Info    : [ 90%] Meshing curve 17 (Extruded)
Info    : [100%] Meshing curve 21 (Extruded)
Info    : Done meshing 1D (Wall 0.000610421s, CPU 0.000313s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 5 (Extruded)
Info    : [ 20%] Meshing surface 14 (Extruded)
Info    : [ 40%] Meshing surface 18 (Extruded)
Info    : [ 50%] Meshing surface 22 (Extruded)
Info    : [ 70%] Meshing surface 26 (Extruded)
Info    : [ 90%] Meshing surface 27 (Extruded)
Info    : Done meshing 2D (Wall 0.00435993s, CPU 0.001783s)
Info    : Meshing

In [7]:
simulation,num_steps = get_rectangular_hit_sim([1,1],[0,0],2,mesh,data_dir)
print(simulation.presses[0].points)
print(simulation.presses[1].points)
print(f"NUM STEPS: {num_steps}")

[06-26 20:02:58][DEBUG] jax_fem: Computing shape function values, gradients, etc.
[06-26 20:02:58][DEBUG] jax_fem: ele_type = HEX8, quad_points.shape = (num_quads, dim) = (8, 3)
[06-26 20:02:58][DEBUG] jax_fem: face_quad_points.shape = (num_faces, num_face_quads, dim) = (6, 4, 3)
[06-26 20:02:58][DEBUG] jax_fem: Done pre-computations, took 0.025093555450439453 [s]
[06-26 20:02:58][INFO] jax_fem: Solving a problem with 729 cells, 1000x3 = 3000 dofs.
/home/timothy/GitHub/NeoForgeServer/Library/create_press.py:19: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot,_ = Rotation.align_vectors([[0.,0.,-1.]],[displacement])


[[ 1.72945  0.37674  2.72768]
 [ 0.72945  0.37674  2.72768]
 [ 0.72945 -0.62326  2.72768]
 [ 1.72945 -0.62326  2.72768]]
[[ 6.22945  4.87674 -0.3473 ]
 [-3.77055  4.87674 -0.3473 ]
 [-3.77055 -5.12326 -0.3473 ]
 [ 6.22945 -5.12326 -0.3473 ]]
NUM STEPS: 66


/home/timothy/GitHub/NeoForgeServer/Library/create_press.py:17: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot,_ = Rotation.align_vectors([[0.,0.,1.]],[displacement])


In [8]:
# import logging
# logging.disable(logging.CRITICAL)
# import warnings
# warnings.filterwarnings("ignore",message="Optimal rotation is not uniquely or poorly defined for the given sets of vectors.")

start = time.time()
simulation.update()
print(f"Update: {time.time()-start}")

print(simulation.presses[0].points)
print(simulation.presses[1].points)

COMBINED_CHECKS
(1000,)
1
COMBINED_DISPLACEMENTS
(1000, 3)
[[0. 0. 0.]]
TABLE
COMBINED_CHECKS
(1000,)
1
COMBINED_DISPLACEMENTS
(1000, 3)
[[ 0.  0. -0.]]
DirichletBCInfo: 0.9595973491668701


[06-26 20:03:10][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[06-26 20:03:10][DEBUG] jax_fem: Start timing
[06-26 20:03:10][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:03:11][DEBUG] jax_fem: Function split_and_compute_cell took 0.7165 seconds
[06-26 20:03:11][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:03:12][DEBUG] jax_fem: Before, res l_2 = 0.005017522244429392
[06-26 20:03:12][DEBUG] jax_fem: Solving linear system with lift solver...
[06-26 20:03:12][DEBUG] jax_fem: Compute and use jacobi preconditioner
[06-26 20:03:13][DEBUG] jax_fem: JAX scipy linear solve res = 4.5503368791072775e-06
[06-26 20:03:13][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:03:13][DEBUG] jax_fem: Function split_and_compute_cell took 0.0873 seconds
[06-26 20:03:13][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:03:13][DEBUG] jax_fem: res l_2 = 4.550336879105281e-06
[06-26 20:03:13][INFO] j

Simulation: 5.1218695640563965
Update: 6.104348659515381
[[ 1.72945  0.37674  2.69768]
 [ 0.72945  0.37674  2.69768]
 [ 0.72945 -0.62326  2.69768]
 [ 1.72945 -0.62326  2.69768]]
[[ 6.22945  4.87674 -0.3473 ]
 [-3.77055  4.87674 -0.3473 ]
 [-3.77055 -5.12326 -0.3473 ]
 [ 6.22945 -5.12326 -0.3473 ]]


In [9]:
for i in range(num_steps-1):
#for i in range(15):
    print(f"STEP: {i+2}")
    
    start = time.time()
    simulation.update()
    print(f"Update: {time.time()-start}")

    #print(simulation.presses[0].points)
    #print(simulation.presses[1].points)
    if simulation.res_vals[-1] > 1:
        print("SIMULATION BLEW UP")
        print(f"RES VAL: {simulation.res_vals[-1]}")
        break

STEP: 2
COMBINED_CHECKS
(1000,)
1
COMBINED_DISPLACEMENTS
(1000, 3)
[[ 0.    0.   -0.03]]
TABLE
COMBINED_CHECKS
(1000,)
1
COMBINED_DISPLACEMENTS
(1000, 3)
[[0. 0. 0.]]
DirichletBCInfo: 0.0622401237487793


[06-26 20:04:01][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[06-26 20:04:01][DEBUG] jax_fem: Start timing
[06-26 20:04:01][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:04:01][DEBUG] jax_fem: Function split_and_compute_cell took 0.0852 seconds
[06-26 20:04:01][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:04:01][DEBUG] jax_fem: Before, res l_2 = 249.14587054777084
[06-26 20:04:01][DEBUG] jax_fem: Solving linear system with lift solver...
[06-26 20:04:01][DEBUG] jax_fem: Compute and use jacobi preconditioner
[06-26 20:04:02][DEBUG] jax_fem: JAX scipy linear solve res = 0.0024702616551566885
[06-26 20:04:02][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:04:02][DEBUG] jax_fem: Function split_and_compute_cell took 0.0807 seconds
[06-26 20:04:02][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:04:02][DEBUG] jax_fem: res l_2 = 25.764357539235878
[06-26 20:04:02][DEBUG] jax_fe

Simulation: 4.825514554977417
Update: 4.889424800872803
STEP: 3
COMBINED_CHECKS
(1000,)
1
COMBINED_DISPLACEMENTS
(1000, 3)
[[ 0.    0.   -0.03]]
TABLE
COMBINED_CHECKS
(1000,)
1
COMBINED_DISPLACEMENTS
(1000, 3)
[[0. 0. 0.]]
DirichletBCInfo: 0.062211036682128906


[06-26 20:04:06][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[06-26 20:04:06][DEBUG] jax_fem: Start timing
[06-26 20:04:06][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:04:06][DEBUG] jax_fem: Function split_and_compute_cell took 0.0898 seconds
[06-26 20:04:06][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:04:06][DEBUG] jax_fem: Before, res l_2 = 251.57143032220912
[06-26 20:04:06][DEBUG] jax_fem: Solving linear system with lift solver...
[06-26 20:04:06][DEBUG] jax_fem: Compute and use jacobi preconditioner
[06-26 20:04:06][DEBUG] jax_fem: JAX scipy linear solve res = 0.0021904106479245246
[06-26 20:04:06][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:04:07][DEBUG] jax_fem: Function split_and_compute_cell took 0.0840 seconds
[06-26 20:04:07][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:04:07][DEBUG] jax_fem: res l_2 = 8.235573076962254
[06-26 20:04:07][DEBUG] jax_fem

Simulation: 3.94360613822937
Update: 4.006743669509888
STEP: 4
COMBINED_CHECKS
(1000,)
2
COMBINED_DISPLACEMENTS
(1000, 3)
[[ 0.       0.      -0.00079]
 [ 0.       0.      -0.03   ]]
TABLE
COMBINED_CHECKS
(1000,)
2
COMBINED_DISPLACEMENTS
(1000, 3)
[[0.    0.    0.   ]
 [0.    0.    0.014]]
DirichletBCInfo: 0.3840610980987549


...

Simulation: 3.6286604404449463
Update: 4.0220770835876465
STEP: 65
COMBINED_CHECKS
(1000,)
35
COMBINED_DISPLACEMENTS
(1000, 3)
[[ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.00442]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.00521]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.0131 ]
 [ 

[06-26 20:08:06][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[06-26 20:08:06][DEBUG] jax_fem: Start timing
[06-26 20:08:06][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:08:06][DEBUG] jax_fem: Function split_and_compute_cell took 0.0858 seconds
[06-26 20:08:07][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:08:07][DEBUG] jax_fem: Before, res l_2 = 1839.2637812060527
[06-26 20:08:07][DEBUG] jax_fem: Solving linear system with lift solver...
[06-26 20:08:07][DEBUG] jax_fem: Compute and use jacobi preconditioner
[06-26 20:08:07][DEBUG] jax_fem: JAX scipy linear solve res = 0.01788858032470202
[06-26 20:08:07][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:08:07][DEBUG] jax_fem: Function split_and_compute_cell took 0.0792 seconds
[06-26 20:08:07][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:08:07][DEBUG] jax_fem: res l_2 = 0.3963979347472917
[06-26 20:08:07][DEBUG] jax_fem:

Simulation: 2.8351376056671143
Update: 3.347818374633789
STEP: 66
COMBINED_CHECKS
(1000,)
36
COMBINED_DISPLACEMENTS
(1000, 3)
[[ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.00111]
 [ 0.       0.      -0.03   ]
 [ 0.       0.      -0.03   ]
 [ 0

[06-26 20:08:10][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[06-26 20:08:10][DEBUG] jax_fem: Start timing
[06-26 20:08:10][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:08:10][DEBUG] jax_fem: Function split_and_compute_cell took 0.0694 seconds
[06-26 20:08:10][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:08:10][DEBUG] jax_fem: Before, res l_2 = 1914.371300005516
[06-26 20:08:10][DEBUG] jax_fem: Solving linear system with lift solver...
[06-26 20:08:10][DEBUG] jax_fem: Compute and use jacobi preconditioner
[06-26 20:08:11][DEBUG] jax_fem: JAX scipy linear solve res = 0.01905724616885439
[06-26 20:08:11][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[06-26 20:08:11][DEBUG] jax_fem: Function split_and_compute_cell took 0.0703 seconds
[06-26 20:08:11][DEBUG] jax_fem: Creating sparse matrix with scipy...
[06-26 20:08:11][DEBUG] jax_fem: res l_2 = 0.5095632496534288
[06-26 20:08:11][DEBUG] jax_fem: 

Simulation: 3.635791301727295
Update: 4.037518501281738


In [10]:
create_gif(data_dir,data_dir)

In [11]:
create_gif(data_dir,data_dir,"animation2.gif",negative=True)

In [2]:
data_dir = 'data/runs/run_009'
data_dir

'data/runs/run_009'

In [15]:
import meshio
vtkMesh = meshio.read('data/runs/run_011/u_066.vtu')
sol_points = vtkMesh.points + simulation.sol_list[0]
print(vtkMesh.points.shape)
print(min(sol_points[:,2]))
lessThan = sol_points[:,2]<-0.3
print(sol_points[lessThan])

(1000, 3)
-0.34801948415520634
[[ 0.09353 -0.03799 -0.3473 ]
 [ 1.96962  0.      -0.3473 ]
 [ 0.3281  -0.01423 -0.3473 ]
 [ 0.5561   0.00487 -0.3473 ]
 [ 0.76694  0.01131 -0.3473 ]
 [ 0.96809  0.0096  -0.3473 ]
 [ 1.15809  0.00851 -0.3473 ]
 [ 1.34343  0.00695 -0.3473 ]
 [ 1.53361  0.00574 -0.3473 ]
 [ 1.73971  0.00467 -0.3473 ]
 [ 2.26043  0.2738  -0.3473 ]
 [ 2.4086   0.52584 -0.34573]
 [-0.14189 -0.30727 -0.34175]
 [-0.20257 -0.47223 -0.30016]
 [ 2.28484 -0.20975 -0.3473 ]
 [ 2.43146 -0.41732 -0.3473 ]
 [ 2.5126  -0.60691 -0.3473 ]
 [ 2.49321 -0.79577 -0.3473 ]
 [ 2.42664 -0.99915 -0.3473 ]
 [ 2.35027 -1.23389 -0.34789]
 [ 0.1163   0.29149 -0.31766]
 [ 0.40296  0.31715 -0.33163]
 [ 0.6853   0.32997 -0.34243]
 [ 0.9598   0.33331 -0.3473 ]
 [ 1.22616  0.3283  -0.3473 ]
 [ 1.48826  0.31322 -0.3473 ]
 [ 1.58205  0.51066 -0.30661]
 [ 1.74674  0.29896 -0.3473 ]
 [ 1.85902  0.50309 -0.3104 ]
 [ 2.00326  0.28855 -0.3473 ]
 [ 2.13401  0.51531 -0.32759]
 [ 0.15631 -0.29251 -0.3473 ]
 [ 0.1045